# Import Required Libraries

In [1]:
import pandas as pd

# Load Required Data

In [2]:
train_df=pd.read_csv("train_data.csv")
passenger_df=pd.read_csv("passengers.csv")

# Calculate Revenue for each Train

In [3]:
#Dictionary to store the extra fare for each class
class_dict={
    'A':50,
    'B':30,
    'C':20,
    'D':10
}

# Join both the tables
com_df=pd.merge(
    left=train_df,
    right=passenger_df,
    how='inner',
    on='Train ID'    
)

#Calculate available seats
com_df['Available Seats']=com_df['Total Seats']-com_df['Number of Tickets']

#Adding the status of the booking
def checkStatus(x:pd.Series):
    if x>0:
        return 'Booked'
    else:
        return 'Cancelled'
com_df['Status']=com_df['Available Seats'].apply(checkStatus)

#Find the passengers whose booking was cancelled
inval_seats=com_df['Available Seats']<0
for pname in com_df.loc[inval_seats,'Passenger Name']:
    print(f"Booking was cancelled for {pname} due to unavailibility of seats")
    
#Change the Available Seats to original value if found invalid
com_df.loc[inval_seats,'Available Seats']+=com_df.loc[inval_seats,'Number of Tickets']

#Change the number of tickets to 0 if found invalid
com_df.loc[inval_seats,'Number of Tickets']=0

#Calculate the revenue according to the class passengers have booked 
def CalculateRevenue(x):
    return x['Number of Tickets']*(x['Total Fare']+class_dict[x['Class']])
com_df['Revenue']=com_df.apply(CalculateRevenue,axis=1)

#Calculate revenue for each train
train_revenue_df=com_df.groupby(['Train ID','Train Name'])['Revenue'].sum().reset_index()

# Create a report 1 and report 2.

In [4]:
#Generate report for the details of the trains
report1=com_df[['Train ID','Train Name','Source Station','Destination Station','Available Seats']]
report1.to_csv('Report-1.csv',index=False)

#Generate report for revenue of each train
train_revenue_df.to_csv('Report-2.csv',index=False)